In [9]:
import pandas as pd
import numpy as np
import pickle
from datetime import datetime
pd.set_option('display.max_columns', None)

# ACS API Queries
Variables: https://www.census.gov/data/developers/data-sets/acs-5year.2016.html#list-tab-1806015614

In [148]:
import requests
import json
df_model = pd.DataFrame()
yr_df_list = []
host = 'https://api.census.gov/data'
years = ['2010','2011','2012','2013','2014','2015','2016','2017','2018','2019','2020','2021','2022','2023']
dataset_acronym = '/acs/acs5'
g = '?get='
variables_v2 = 'NAME,B02001_001E,B19051_001E,B19057_002E,B19058_002E,C16002_002E,C24050_001E,C24050_002E,C24050_003E,C24050_006E,C24050_009E,C24050_010E,C24050_011E,C24050_012E,B07001_049E,B07001_033E,B08006_008E,B08006_002E,B08006_014E,B08006_015E,B08006_017E,B02001_002E,B02001_003E,B02001_004E,B02001_005E,B02001_006E,B03001_003E,B06009_005E,B06009_006E,B05001_002E,B05001_005E,B05001_006E,B07001_017E,B07001_065E,B19013_001E,B25064_001E,B25035_001E,B25039_001E,B01002_001E,B25036_013E,B25077_001E,B25040_004E,B25040_008E'
variables_v1 = 'NAME,B02001_001E,B19051_001E,B19057_002E,B19058_002E,B16002_002E,C24050_001E,C24050_002E,C24050_003E,C24050_006E,C24050_009E,C24050_010E,C24050_011E,C24050_012E,B07001_049E,B07001_033E,B08006_008E,B08006_002E,B08006_014E,B08006_015E,B08006_017E,B02001_002E,B02001_003E,B02001_004E,B02001_005E,B02001_006E,B03001_003E,B06009_005E,B06009_006E,B05001_002E,B05001_005E,B05001_006E,B07001_017E,B07001_065E,B19013_001E,B25064_001E,B25035_001E,B25039_001E,B01002_001E,B25036_013E,B25077_001E,B25040_004E,B25040_008E'

location = '&for=TRACT:*&in=state:36%county:005,047,061,081,085'
usr_key = f"&key=332de56715b7f43a68ee347091c29afad759be55"

for year in years:
    print(year)
    if year in ['2010','2011','2012','2013','2014','2015','2022']:
        variables = variables_v1
    else:
        variables = variables_v2
    query_url = f"{host}/{year}{dataset_acronym}{g}{variables}{location}{usr_key}"
    
    # Use requests package to call out to the API
    response = requests.get(query_url)

    ## Rename columns for clarity
    table_lists = json.loads(response.text)
    df = pd.DataFrame(table_lists[1:], columns=table_lists[0])
    df['year'] = year

    col_names = {"B02001_001E":"total_pop",
                 "B02001_002E":"percent_white",
                 "B02001_003E":"percent_black",
                 "B02001_004E":"percent_native",
                 "B02001_005E":"percent_asian",
                 "B02001_006E":"percent_pacific",
                 "B03001_003E":"percent_latino",
                 "C24050_001E":"total_num_industry",  # total CIVILIAN EMPLOYED POPULATION 16 YEARS AND OVER
                 "C24050_002E":"percent_work_agriculture",  # Total:!!Agriculture, forestry, fishing and hunting, and mining:
                 "C24050_003E":"percent_work_construction",
                 "C24050_006E":"percent_work_retail",
                 "C24050_009E":"percent_work_finance",
                 "C24050_010E":"percent_work_stem",  # Professional, scientific, and management, and administrative, and waste management services:
                 "C24050_011E":"percent_work_edu_health",  # al:!!Educational services, and health care and social assistance:
                 "C24050_012E":"percent_work_art",  # Arts, entertainment, and recreation, and accommodation and food services:
                 "B06009_005E":"percent_bachelors",  # num ppl with bachelors degree
                 "B06009_006E":"percent_grad",
                 "B18140_002E":"percent_disabled",  # num ppl With a disability:
                 "B05001_002E":"percent_born_citizen",
                 "B05001_005E":"percent_naturalized_citizen",
                 "B05001_006E":"percent_not_citizen",
                 "B07001_017E":"percent_same_house_1yr",  # Estimate!!Total:!!Same house 1 year ago:
                 "B07001_065E":"percent_out_of_state",  # Estimate!!Total:!!Moved from different state: (in past year)
                 "B07001_049E":"percent_out_of_county",  # Estimate!!Total:!!Moved from different county within same state (past year)
                 "B07001_033E":"percent_moved_within_county",  # !Total:!!Moved within same county (past year)
                 "B08006_008E":"percent_public_transit",  # !!Total:!!Public transportation to work (excluding taxicab)
                 "B08006_002E":"percent_drive_commute",  # !!Total:!!drive to work
                 "B08006_014E":"percent_bike_commute",
                 "B08006_015E":"percent_walk_commute",  # num ppl walk to work
                 "B08006_017E":"percent_wfh",  # num ppl work from home
                 "B19013_001E":"med_income",  # Estimate!!Median household income in the past 12 months (in 2019 inflation-adjusted dollars)
                 "B25064_001E":"med_rent_acs",  # Estimate!!Median gross rent
                 "B25035_001E":"med_yr_built",  # !Median year structure built
                 "B01002_001E":"med_age",
                 "B25036_013E":"percent_hh_rented",  # num hh renter occupied
                 "B25077_001E":"med_value",  # median home value
                 "B25039_001E":"med_yr_moved_in",  # Estimate!!Median year householder moved into unit rent or own
                 "B19051_001E":"total_num_hh",
                 "C16002_002E":"percent_hh_english",  # num hh speaking english only  #### not in 2016
                 "B16002_002E":"percent_hh_english",  # num hh speaking english only
                 "B19058_002E":"percent_hh_snap",  # Estimate!!Total:!!Household received Food Stamps/SNAP in the past 12 months:
                 "B19057_002E":"percent_hh_income_assist",  # PUBLIC ASSISTANCE INCOME IN THE PAST 12 MONTHS FOR HOUSEHOLDS
                 "B25040_004E":"percent_hh_electric",  # Estimate!!Total:!!Electricity
                 "B25040_008E":"percent_hh_solar"}  # Estimate!!Total:!!Solar energy
        
    df.rename(columns= col_names, inplace=True)

    # random formatting
    df['med_yr_built'] = df['med_yr_built'].replace("-",np.nan)
    df['med_yr_built'] = df['med_yr_built'].apply(lambda x: str(x).replace("-",""))
    df['med_yr_built'] = df['med_yr_built'].apply(lambda x: str(x).replace("+",""))
    df['med_yr_moved_in']= df['med_yr_moved_in'].replace("-",np.nan)
    df['med_yr_moved_in'] = df['med_yr_moved_in'].apply(lambda x: str(x).replace("+",""))

    # change year into number of years from present
    current_year = datetime.now().year
    df['med_hh_age'] = current_year - df['med_yr_built'].astype(float)
    df['med_hh_tenure'] = current_year - df['med_yr_moved_in'].astype(float)
    
    # replace all outliers with nan
    # columns with very large positive outliers
    for col in ['med_hh_tenure','med_yr_built']:
        df[col] = np.where(df[col].astype(float)>66666868, np.nan, df[col].astype(float))
    df['med_income'] = df['med_income'].astype(float)

    # columns with very small negative outliers
    for col in ['percent_work_agriculture','percent_work_construction', 'percent_work_retail', 'percent_work_finance', 'percent_work_stem', 'percent_work_edu_health','percent_work_art','med_income', 'med_rent_acs', 'med_age', 'med_value', 'med_hh_age','total_num_industry','med_yr_moved_in']:
        df[col] = np.where(df[col].astype(float)<0, np.nan, df[col].astype(float))

    # change house age of 2024 to zero
    df.loc[df['med_hh_age']>1000, 'med_hh_age'] = 2025-df.loc[df['med_hh_age']>1000, 'med_hh_age']

    
    # change counts into percentages
    for col in ['percent_out_of_county',
       'percent_moved_within_county', 'percent_public_transit',
       'percent_drive_commute', 'percent_bike_commute', 'percent_walk_commute',
       'percent_wfh','percent_white', 'percent_black',
       'percent_native', 'percent_asian', 'percent_pacific', 'percent_latino','percent_bachelors', 'percent_grad',
       'percent_born_citizen', 'percent_naturalized_citizen',
       'percent_not_citizen', 'percent_same_house_1yr', 'percent_out_of_state']:
        df[col] = df[col].astype(float)/df['total_pop'].astype(float) 
    for col in ['percent_work_agriculture',
       'percent_work_construction', 'percent_work_retail',
       'percent_work_finance', 'percent_work_stem', 'percent_work_edu_health',
       'percent_work_art']:
        df[col] = df[col].astype(float)/df['total_num_industry'].astype(float)
    for col in ['percent_hh_income_assist',
       'percent_hh_snap', 'percent_hh_english','percent_hh_rented','percent_hh_electric', 'percent_hh_solar']:
        df[col] = df[col].astype(float)/df['total_num_hh'].astype(float)

    df['full_tract'] = df.county + df.tract
    
    df.drop(["med_yr_moved_in","med_yr_built","state","total_num_hh","total_num_industry","county","tract"], axis=1, inplace=True)
    yr_df_list.append(df)  


2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023


In [149]:
# loop through each df and join to get full tracts series
all_tracts = yr_df_list[0]
for yr_df in yr_df_list[1:]:
    all_tracts = all_tracts[['full_tract']].merge(yr_df[['full_tract']], how='outer').drop_duplicates()

# loop through again and join full tracts series with each df
to_concat=[]
for yr_df in yr_df_list:
    yr_df = yr_df.merge(all_tracts, how='outer')
    # fill in the null years
    yr_df['year'] = yr_df['year'].ffill()
    to_concat.append(yr_df)

# concat all df's
df_model = pd.concat(to_concat)


In [150]:
# change datatypes
df_model['med_value'] = df_model['med_value'].astype(float)
df_model['med_age'] = df_model['med_age'].astype(float)
df_model['med_rent_acs'] = df_model['med_rent_acs'].astype(float)
df_model['med_income'] = df_model['med_income'].astype(float)
df_model['total_pop'] = df_model['total_pop'].astype(float)

# add GEOID column for mapping
df_model['GEOID'] = '36'+df_model['full_tract']


In [151]:
df_model.describe()

,total_pop,percent_hh_income_assist,percent_hh_snap,percent_hh_english,percent_work_agriculture,percent_work_construction,percent_work_retail,percent_work_finance,percent_work_stem,percent_work_edu_health,percent_work_art,percent_out_of_county,percent_moved_within_county,percent_public_transit,percent_drive_commute,percent_bike_commute,percent_walk_commute,percent_wfh,percent_white,percent_black,percent_native,percent_asian,percent_pacific,percent_latino,percent_bachelors,percent_grad,percent_born_citizen,percent_naturalized_citizen,percent_not_citizen,percent_same_house_1yr,percent_out_of_state,med_income,med_rent_acs,med_age,percent_hh_rented,med_value,percent_hh_electric,percent_hh_solar,med_hh_age,med_hh_tenure
count,30979.000000,30096.000000,30096.000000,30096.000000,30140.000000,30140.000000,30140.000000,30140.000000,30140.000000,30140.000000,30140.000000,30223.000000,30223.000000,30223.000000,30223.000000,30223.000000,30223.000000,30223.000000,30223.000000,30223.000000,30223.000000,30223.000000,30223.000000,30223.000000,30223.000000,30223.000000,30223.000000,30223.000000,30223.000000,30223.000000,30223.000000,29854.000000,29710.000000,30166.000000,30096.000000,2.747400e+04,30096.000000,30096.000000,29943.000000,29882.000000
mean,3796.011266,0.045349,0.203340,0.497391,0.001108,0.054130,0.096438,0.088425,0.122926,0.280453,0.099684,0.020693,0.057603,0.240723,0.135335,0.004874,0.042020,0.027971,0.419265,0.254878,0.004479,0.138697,0.000576,0.267907,0.147690,0.102077,0.592849,0.210117,0.161320,0.885887,0.013720,66947.918671,1465.756244,37.319714,0.409783,6.384007e+05,0.091562,0.000507,64.773269,19.866107
std,2177.840906,0.045000,0.157606,0.220496,0.005412,0.046088,0.047764,0.061516,0.069163,0.100740,0.064087,0.032345,0.043068,0.097153,0.091137,0.009506,0.052876,0.035510,0.296976,0.298249,0.012342,0.167104,0.006415,0.226305,0.090127,0.095554,0.153110,0.103067,0.098014,0.077149,0.025174,34839.481436,510.552872,6.947215,0.360797,3.152925e+05,0.108088,0.003733,27.023834,5.113665
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2499.000000,231.000000,1.900000,0.000000,9.999000e+03,0.000000,0.000000,0.000000,3.000000
25%,2311.000000,0.013004,0.080411,0.315179,0.000000,0.021731,0.064072,0.049235,0.076116,0.210351,0.055944,0.003792,0.028427,0.174580,0.063119,0.000000,0.013372,0.007119,0.139745,0.019841,0.000000,0.021333,0.000000,0.092324,0.082892,0.035827,0.484472,0.127251,0.090909,0.856664,0.000000,42500.000000,1161.000000,32.800000,0.000000,4.382000e+05,0.028673,0.000000,61.000000,16.000000
50%,3482.000000,0.032258,0.163224,0.501424,0.000000,0.045590,0.091442,0.075598,0.107873,0.275291,0.087604,0.012868,0.050197,0.234739,0.112792,0.000000,0.027549,0.016487,0.381617,0.098377,0.000000,0.071549,0.000000,0.186314,0.127891,0.071510,0.591422,0.199155,0.142747,0.900990,0.005180,61071.000000,1389.000000,36.700000,0.411596,5.750000e+05,0.056800,0.000000,73.000000,20.000000
75%,4877.000000,0.064449,0.290485,0.687449,0.000000,0.075194,0.122960,0.110321,0.154073,0.344781,0.130918,0.027701,0.077365,0.300480,0.195550,0.006083,0.049195,0.034874,0.685879,0.439132,0.004168,0.195778,0.000000,0.397182,0.192312,0.132567,0.710287,0.279432,0.213047,0.934723,0.016637,83125.000000,1698.000000,41.100000,0.748372,7.653750e+05,0.109976,0.000000,86.000000,23.000000
max,29256.000000,1.000000,1.000000,1.000000,0.250000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.169231,1.000000,0.518519,1.000000,1.000000,0.571429,1.000000,0.557377,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.500000,250001.000000,3501.000000,94.000000,1.000000,2.000001e+06,1.000000,0.239617,87.000000,56.000000


In [153]:
# df_model.to_parquet("Data/Cleaned/ACS.parquet")